## This notebook is for creating and testing different models on classifying a salary range for a job posting

In [73]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer

In [155]:
df = pd.read_csv(os.path.dirname(os.getcwd())+"\\EDA\\clean_data.csv")
df["loc_state"] = df["Location"].apply(lambda x: x.split(',')[1].strip())
# Take only rows if there are more than 10 of the same target label. We have to do this because otherwise the chance of getting all unique targets in train or validation set is too high.
sub_df = df[(df.groupby('Salary Estimate')["Salary Estimate"].transform('count')>5) \
    and (df.groupby('Industry')["Industry"].transform('count')>10) \
    and (df.groupby('new_title')["new_title"].transform('count')>10)\
    and (df.groupby('loc_state')["loc_state"].transform('count')>10)].copy() 
df.head()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [111]:
df = df.dropna(axis=0) # For non drop all missing values

In [112]:
cols_to_keep = ["Rating",'Size','Type of ownership','Industry','Sector','Revenue','new_title','loc_city','hq_city']
X = df[cols_to_keep]
y = df["Salary Estimate"]

In [154]:
df["loc_state"].value_counts().head(50)

# X["new_title"].value_counts()

CA          626
TX          394
NY          345
IL          164
PA          114
AZ           97
NC           89
CO           88
NJ           86
WA           54
VA           48
OH           35
UT           33
FL           27
IN           23
DE           11
Arapahoe      8
GA            4
KS            3
SC            3
Name: loc_state, dtype: int64

In [113]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Train + val & test

numeric_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = [c for c in X.columns if c not in numeric_features]
columnTransformer = ColumnTransformer(
    transformers=[
        ('cat', OrdinalEncoder(), categorical_features)])

X_train = columnTransformer.fit_transform(X_train)
X_test = columnTransformer.fit_transform(X_test)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42) # Train & val

cat_idx = [X.columns.get_loc(col) for col in categorical_features]